# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

https://towardsdatascience.com/building-a-disaster-response-web-application-4066e6f90072

In [1]:
# import libraries pak smaze
import pandas as pd
import numpy as np
import sys # is not necessary

from sqlalchemy import create_engine

import pickle

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier




# from flask_sqlalchemy import SQLAlchemy https://pythonbasics.org/flask-sqlalchemy/#Flask-SQLAlchemy

In [2]:
# import nltk and sklearn libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download(['punkt', 'wordnet','stopwords'])
import sys # asi není potřeba


from sklearn.datasets import make_multilabel_classification

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
##########################################################################
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import GridSearchCV



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db').connect()  
df = pd.read_sql_table('InsertTableName', engine)
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [4]:
# removes rows which are null in message column
df.dropna(subset=['message'],axis=0,inplace=True)

# This machine pipeline should take in the message column as input 
X_pandas = df.message
# creates numpy
X = X_pandas.values

Y_pandas= df.drop(columns = ['id','message','original','genre'], axis = 1)
Y = Y_pandas.values

categories_columns = Y_pandas.columns

In [5]:
X.shape,Y.shape

((25993,), (25993, 36))

In [6]:
(type(X),type(Y))
# df.info()

(numpy.ndarray, numpy.ndarray)

In [7]:
Y_pandas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25993 entries, 0 to 25992
Data columns (total 36 columns):
related                   25993 non-null int64
request                   25993 non-null int64
offer                     25993 non-null int64
aid_related               25993 non-null int64
medical_help              25993 non-null int64
medical_products          25993 non-null int64
search_and_rescue         25993 non-null int64
security                  25993 non-null int64
military                  25993 non-null int64
child_alone               25993 non-null int64
water                     25993 non-null int64
food                      25993 non-null int64
shelter                   25993 non-null int64
clothing                  25993 non-null int64
money                     25993 non-null int64
missing_people            25993 non-null int64
refugees                  25993 non-null int64
death                     25993 non-null int64
other_aid                 25993 non-null int6

In [8]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

### 2. Write a tokenization function to process your text data

In [9]:
""" Origin 

def tokenize(text):
    pass
"""
def tokenize1(text):
    
    # if text is np.nan
    
    #only low case
    #text = text.v
    
    # get rid nonalphanumeric characters
    # https://regexr.com/3a8p3 - testing possible     
    not_alphanumeric_reg = r"[^a-zA-Z0-9]"
    text = re.sub(not_alphanumeric_reg, " ", text)
    
    
    #tokenize 
    tokens = word_tokenize(text)
    
    
    # init a remove stopwords
    stop_words = stopwords.words("english")
    #vystup = [lemmatizer.lemmatize(p) for p in tokens if not p in stop_words]
    
    #lematizer instance
    lemmatizer = WordNetLemmatizer()
    
    #stemmer instance 
    stemmer = PorterStemmer()
    
    
       
    

    clean_tokens = []
    for tok in tokens:
        if tok not in stop_words:
            # clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            noun_lemmatize_tok = lemmatizer.lemmatize(tok)
            verb_lemmatize_tok = lemmatizer.lemmatize(noun_lemmatize_tok,pos="v")
            stem_lem_token = stemmer.stem(verb_lemmatize_tok)

            new_tok = stem_lem_token.lower().strip() 
            clean_tokens.append(new_tok)

    return clean_tokens
    pass

In [10]:
# [p.split(' ') for p in X_pandas.head().values]
#example tokenize1
[tokenize1(p) for p in X_pandas.head().values]


[['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pa', 'haiti'],
 ['is', 'hurrican'],
 ['look', 'someon', 'name'],
 ['un',
  'report',
  'leogan',
  '80',
  '90',
  'destroy',
  'onli',
  'hospit',
  'st',
  'croix',
  'function',
  'need',
  'suppli',
  'desper'],
 ['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight']]

In [11]:
   
def tokenize2(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
   
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens    
    
    pass

In [12]:
#example tokenize2
[tokenize2(p) for p in X_pandas.head().values]

[['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'],
 ['hurricane'],
 ['looking', 'someone', 'name'],
 ['un',
  'report',
  'leogane',
  '80',
  '90',
  'destroyed',
  'hospital',
  'st',
  'croix',
  'functioning',
  'need',
  'supply',
  'desperately'],
 ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']]

tokenize1 method and tokenize2 method are giving same output

In [13]:
"""
# zdrojove
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

    # train classifier
    pipeline.fit(X_train, y_train)

    # predict on test data
    y_pred = pipeline.predict(X_test)

    # display results
    display_results(y_test, y_pred)


main()
"""

'\n# zdrojove\ndef display_results(y_test, y_pred):\n    labels = np.unique(y_pred)\n    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)\n    accuracy = (y_pred == y_test).mean()\n\n    print("Labels:", labels)\n    print("Confusion Matrix:\n", confusion_mat)\n    print("Accuracy:", accuracy)\n\n\ndef main():\n    X, y = load_data()\n    X_train, X_test, y_train, y_test = train_test_split(X, y)\n\n    pipeline = Pipeline([\n        (\'vect\', CountVectorizer(tokenizer=tokenize)),\n        (\'tfidf\', TfidfTransformer()),\n        (\'clf\', RandomForestClassifier())\n    ])\n\n    # train classifier\n    pipeline.fit(X_train, y_train)\n\n    # predict on test data\n    y_pred = pipeline.predict(X_test)\n\n    # display results\n    display_results(y_test, y_pred)\n\n\nmain()\n'

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
def model_pipeline():
    # TfidfVectorizer is the combination of CountVectorizer + TfidfTransformer.
    pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize1)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10)))
                        ])
    return pipeline
pass

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
from sklearn.model_selection import GridSearchCV, train_test_splitfrom sklearn
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3) 

In [16]:
print("""
\t X: {}
\t Y: {}
\t X_train: {}
\t X_test: {}
\t Y_train: {}
\t Y_test: {}""".format(X.shape,Y.shape,X_train.shape, X_test.shape, Y_train.shape , Y_test.shape)
     )



	 X: (25993,)
	 Y: (25993, 36)
	 X_train: (18195,)
	 X_test: (7798,)
	 Y_train: (18195, 36)
	 Y_test: (7798, 36)


In [17]:
X_test[1]

'There was a lot of debris floating in the water, including bar stools and other pieces of furniture from Macaronis Resort, he said.'

In [18]:
Y_test[1]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
print("""
\t X: {}
\t Y: {}
\t X_train: {}
\t X_test: {}
\t Y_train: {}
\t Y_test: {}""".format(type(X),type(Y),type(X_train),type(X_test),type(Y_train),type(Y_test)
                       ))


	 X: <class 'numpy.ndarray'>
	 Y: <class 'numpy.ndarray'>
	 X_train: <class 'numpy.ndarray'>
	 X_test: <class 'numpy.ndarray'>
	 Y_train: <class 'numpy.ndarray'>
	 Y_test: <class 'numpy.ndarray'>


In [20]:
Y_test[:, 5]

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
'''# Troubleshooting
from sklearn.feature_extraction.text import CountVectorizer

# initialize count vectorizer object
vect = CountVectorizer(tokenizer=tokenize1)# get counts of each token (word) in text data

# get counts of each token (word) in text data
corpus = X
X = vect.fit_transform(corpus)

# view token vocabulary and counts
#vect.vocabulary_

# convert sparse matrix to numpy array to view
X

'''



'# Troubleshooting\nfrom sklearn.feature_extraction.text import CountVectorizer\n\n# initialize count vectorizer object\nvect = CountVectorizer(tokenizer=tokenize1)# get counts of each token (word) in text data\n\n# get counts of each token (word) in text data\ncorpus = X\nX = vect.fit_transform(corpus)\n\n# view token vocabulary and counts\n#vect.vocabulary_\n\n# convert sparse matrix to numpy array to view\nX\n\n'

In [22]:
# train classifier
model = model_pipeline()
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
# predict on test data moje
from sklearn.metrics import classification_report

Y_predict = model.predict(X_test)



print(classification_report(Y_test, Y_predict, target_names=categories_columns))



                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      5966
               request       0.79      0.44      0.57      1359
                 offer       0.00      0.00      0.00        37
           aid_related       0.74      0.61      0.67      3250
          medical_help       0.58      0.08      0.14       617
      medical_products       0.64      0.08      0.14       356
     search_and_rescue       0.50      0.02      0.04       227
              security       0.20      0.01      0.01       140
              military       0.57      0.12      0.20       258
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.25      0.39       467
                  food       0.82      0.61      0.70       881
               shelter       0.79      0.32      0.46       681
              clothing       0.68      0.21      0.33        98
                 money       0.40      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [24]:
from sklearn.utils.multiclass import type_of_target
# Test the model and print the classification report for each of the 36 categories.
def performance(model, X_test, Y_test):
    Y_predict = model.predict(X_test)
    # type_of_target([0.1, 0.6])
    print("""
        \t Y_predict type: {}
        \t Y_predict shape: {}
        \t Y_predict: {}
        \t Y_test: {}""".format(type(Y_predict),Y_predict.shape,type_of_target(Y_predict),type_of_target(Y_test)                            ))
    
    print(classification_report(Y_test, Y_predict, target_names=categories_columns))

    '''
    for i, col in enumerate(Y_test):
        print(col)
        
        print("""
        \t i=  {}
        \t col=  {}""".format(i,col))
        
        
        # print(classification_report(Y_test[col], Y_predict[:, i]))
        print(classification_report(Y_test[col], Y_predict[col]))
    '''


'''




print(classification_report(Y_test, Y_predict, target_names=categories_columns))

'''







'\n\n\n\n\nprint(classification_report(Y_test, Y_predict, target_names=categories_columns))\n\n'

In [25]:
performance(model, X_test, Y_test)


        	 Y_predict type: <class 'numpy.ndarray'>
        	 Y_predict shape: (7798, 36)
        	 Y_predict: multilabel-indicator
        	 Y_test: multilabel-indicator
                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      5966
               request       0.79      0.44      0.57      1359
                 offer       0.00      0.00      0.00        37
           aid_related       0.74      0.61      0.67      3250
          medical_help       0.58      0.08      0.14       617
      medical_products       0.64      0.08      0.14       356
     search_and_rescue       0.50      0.02      0.04       227
              security       0.20      0.01      0.01       140
              military       0.57      0.12      0.20       258
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.25      0.39       467
                  food       0.82      0.61      0.70       8

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
# https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html





In [27]:
# estimator.get_params() not relevant for 

In [28]:

# %%script false  Lines I don't want to run - skipp cells
'''
parameters = {#'clf__estimator__n_estimators': (5 ,10),
             
             'vect__ngram_range': ((1, 1), (1, 2)),
             'tfidf__use_idf': (True, False),
             'clf__n_estimators': [50, 100, 200]
             }


parameters={'clf__max_features':[0.3, 0.5, 0.7],

            'clf__min_samples_leaf':[1, 2, 3]

            #'clf__max_depth':  [None]

           }
'''

'''cv = GridSearchCV(pipeline, param_grid=params)

cv.fit(X_train,y_train).
'''

# Hyperparameters grid
parameters = {  
              'tfidf__use_idf': (True, False), # asi tento 58dek smazat
              'clf__estimator__n_estimators': [5, 10, 20], 
              'clf__estimator__min_samples_split': [2,3, 4]} 




# initialize 
cv = GridSearchCV(model_pipeline(), param_grid=parameters, verbose=1)
# cv = GridSearchCV(model_pipeline())


for param in cv.get_params().keys():
    print(param)

# fit
cv.fit(X_train, Y_train)
cv.best_params_

cv
error_score
estimator__memory
estimator__steps
estimator__vect
estimator__tfidf
estimator__clf
estimator__vect__analyzer
estimator__vect__binary
estimator__vect__decode_error
estimator__vect__dtype
estimator__vect__encoding
estimator__vect__input
estimator__vect__lowercase
estimator__vect__max_df
estimator__vect__max_features
estimator__vect__min_df
estimator__vect__ngram_range
estimator__vect__preprocessor
estimator__vect__stop_words
estimator__vect__strip_accents
estimator__vect__token_pattern
estimator__vect__tokenizer
estimator__vect__vocabulary
estimator__tfidf__norm
estimator__tfidf__smooth_idf
estimator__tfidf__sublinear_tf
estimator__tfidf__use_idf
estimator__clf__estimator__bootstrap
estimator__clf__estimator__class_weight
estimator__clf__estimator__criterion
estimator__clf__estimator__max_depth
estimator__clf__estimator__max_features
estimator__clf__estimator__max_leaf_nodes
estimator__clf__estimator__min_impurity_decrease
estimator__clf__estimator__min_impurity_split
esti

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 66.6min finished


{'clf__estimator__min_samples_split': 3,
 'clf__estimator__n_estimators': 20,
 'tfidf__use_idf': False}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:

Y_pred = cv.predict(X_test)
    
Y_pred_df = pd.DataFrame(Y_pred, columns = categories_columns)
Y_test_df = pd.DataFrame(Y_test, columns = categories_columns)
    
for i in range(36):
        
    print('Category: {}'
          "\n\n\n classification_report: {}".format(categories_columns[i],
          classification_report(Y_test_df.iloc[:,i], Y_pred_df.iloc[:,i])))

Category: related


 classification_report:              precision    recall  f1-score   support

          0       0.70      0.41      0.52      1832
          1       0.84      0.94      0.89      5966

avg / total       0.81      0.82      0.80      7798

Category: request


 classification_report:              precision    recall  f1-score   support

          0       0.90      0.98      0.94      6439
          1       0.81      0.49      0.61      1359

avg / total       0.88      0.89      0.88      7798

Category: offer


 classification_report:              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7761
          1       0.00      0.00      0.00        37

avg / total       0.99      1.00      0.99      7798

Category: aid_related


 classification_report:              precision    recall  f1-score   support

          0       0.79      0.82      0.80      4548
          1       0.73      0.69      0.71      3250

avg / total     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import AdaBoostClassifier
# Improve the model via pipeline+.
pipeline1 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])
# Train the  pipeline1. 
pipeline1.fit(X_train, Y_train)
# Check the performance of the adjusted model.
performance(pipeline1, X_test, Y_test)


parameters1 = { 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [5, 10],
              'clf__estimator__learning_rate': [1,2] }
cv1 = GridSearchCV(pipeline1, param_grid=parameters1)
cv1.fit(X_train, Y_train)
performance(cv1, X_test, Y_test)







        	 Y_predict type: <class 'numpy.ndarray'>
        	 Y_predict shape: (7798, 36)
        	 Y_predict: multilabel-indicator
        	 Y_test: multilabel-indicator
                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5966
               request       0.00      0.00      0.00      1359
                 offer       0.00      0.00      0.00        37
           aid_related       0.49      0.05      0.09      3250
          medical_help       0.00      0.00      0.00       617
      medical_products       0.00      0.00      0.00       356
     search_and_rescue       0.00      0.00      0.00       227
              security       0.00      0.00      0.00       140
              military       0.00      0.00      0.00       258
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       467
                  food       0.00      0.00      0.00       8

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



        	 Y_predict type: <class 'numpy.ndarray'>
        	 Y_predict shape: (7798, 36)
        	 Y_predict: multilabel-indicator
        	 Y_test: multilabel-indicator
                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5966
               request       0.00      0.00      0.00      1359
                 offer       0.00      0.00      0.00        37
           aid_related       0.00      0.00      0.00      3250
          medical_help       0.00      0.00      0.00       617
      medical_products       0.00      0.00      0.00       356
     search_and_rescue       0.00      0.00      0.00       227
              security       0.00      0.00      0.00       140
              military       0.00      0.00      0.00       258
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       467
                  food       0.00      0.00      0.00       8

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [35]:
#
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    # confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    # https://stackoverflow.com/questions/46953967/multilabel-indicator-is-not-supported-for-confusion-matrix
    # .values.argmax(axis=1)
    confusion_mat = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1), labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


display_results(Y_test, Y_pred)







Labels: [0 1]
Confusion Matrix:
 [[7753   12]
 [   0    0]]
Accuracy: 0.947447351173


### 9. Export your model as a pickle file

In [40]:
pickle_file_name = 'pickle.file1.p'
pickle_filepath = ""
handle_file = open('pickle.file1.p', 'wb')
pickle.dump(cv, handle_file)

# close the file
handle_file.close() 


In [41]:
# alternative pickle
with open('pickle.file1.p', 'wb') as handle_file:
    # Pickle the 'cv' dictionary using the highest protocol available.
    pickle.dump(cv, handle_file)
    
# close the file
handle_file.close() 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# learn https://github.com/andreigalanciuc/DATA-ENGINEERING-DISASTER-MESSAGES
# https://towardsdatascience.com/doing-meaningful-work-with-machine-learning-classify-disaster-messages-436bfe9ec42